Code from which a script can be created for EIA data ingest

TO DO:  Connect this with Maya's credentials file created in the set up script

In [1]:
# REMOVE THE KEY FOR ANY PUBLIC CODE!
# Right now this code is set up assuming this variable is previously defined in the Python environment:
# EIA_API_KEY = ''

In [2]:
# Imports
import os, requests, numpy as np, pandas as pd

First download the list of states

In [3]:
# EIA list of states is here: http://www.eia.gov/opendata/qb.php?category=1017
category_states = '1017'
states_url = 'http://api.eia.gov/category/?api_key=' + EIA_API_KEY + '&category_id=' + category_states
states = requests.get(states_url)

** Do we need to insert some error code for what happens if the status code is no good?  How we manage the error will have to be compatible with how we set up the overall code for the project.

In [4]:
# Create a variable to hold the status code
states_status = states.status_code
# This code will return 200 as an integer if everything downloaded properly
if states_status != 200:
    print("The list of power plants did not download properly. Please run this script again.")
else:
    pass

In [5]:
states_dict = states.json()
states_json = pd.io.json.json_normalize(states_dict['category'])
states_df = pd.DataFrame(states_json['childcategories'][0], columns = ["name", 'category_id'])
states_df.head()

,name,category_id
0,Alabama,902931
1,Alaska,902932
2,Arizona,902933
3,Arkansas,902934
4,California,902935


Is there any need to create separate scripts or keep this all together?

For each state, download the list of all power plants in that state

### ER Diagram: PLANTS
Dataframe created in this code can be used to create the "Plants" table in the ER diagram

In [6]:
# Create empty dataframe to hold the list of all power plants
# Columns = 1) Plant name; 2) Category_id for calling that plant's data via the API
columns = ["name", 'category_id']
all_plants_df = pd.DataFrame(columns = columns)


# Iterate over the category_id value for each state in the states_df dataframe
for state in states_df.itertuples():
    # Isolate the category_id for each state
    state_id = str(state[2])
    # Download the plant data
    plants_url = 'http://api.eia.gov/category/?api_key=' + EIA_API_KEY + '&category_id=' + state_id
    plants = requests.get(plants_url)
    # Turn the downloaded data into a dataframe
    plants_dict = plants.json()
    plants_json = pd.io.json.json_normalize(plants_dict['category'])
    plants_df = pd.DataFrame(plants_json['childcategories'][0], columns = ["name", 'category_id'])
    # Append each state's list of plants to the master all plants datagrame
    temp_df = all_plants_df.append(plants_df, ignore_index = True)
    all_plants_df = temp_df


In [12]:
# Fix names to remove the numbers in parentheses that have indeterminate meaning
plants_name_temp1 = all_plants_df["name"].str.split(')').str.get(1)
plants_name_temp2 = pd.DataFrame(plants_name_temp1.str.split('(').str.get(0), columns = ["name"])
# plants_name_temp3 = pd.DataFrame(plants_name_temp2)
final_plants_df = pd.concat([plants_name_temp2, all_plants_df["category_id"]], axis=1)
final_plants_df

,name,category_id
0,Bankhead Dam,1024
1,Barry,1025
2,Walter Bouldin Dam,1026
3,Gadsden,1027
4,Gorgas,1028
5,Greene County,1030
6,H Neely Henry Dam,1031
7,Holt Dam,1032
8,Jordan Dam,1033
9,Logan Martin Dam,1034


NEXT: Use the category IDs for each plant in the all_plants_df dataframe to isolate out plants with solar power

### ER Diagram: Solar Locations
This code will create the ID and include the plant ID for the Solar Locations table

In [46]:
# for viewing all data in the dataframe
pd.set_option('display.max_colwidth', -1)

In [54]:
# Create empty dataframe to hold the list of all solar power locations
columns = ["name", 'series_id']
solar_df = pd.DataFrame(columns = columns)


# Iterate over each category_id value for each plant in the final_plants_df dataframe
for plant in final_plants_df.itertuples():
    # Isolate the category_id for each state
    plant_id_int = int(plant[2])
    plant_id = str(plant_id_int)
    # Download the plant data
    series_url = 'http://api.eia.gov/category/?api_key=' + EIA_API_KEY + '&category_id=' + plant_id
    series = requests.get(series_url)
    # Turn the downloaded data into a dataframe
    series_dict = series.json()
    series_json = pd.io.json.json_normalize(series_dict['category'])
    series_df = pd.DataFrame(series_json['childseries'][0], columns = ["name", 'series_id'])
    solar_filter1 = pd.DataFrame(series_df[series_df["name"].str.contains("solar") == True])
    solar_filter2 = pd.DataFrame(solar_filter1[solar_filter1["name"].str.startswith("Net generation") == True])
    solar_filter3 = pd.DataFrame(solar_filter2[solar_filter2["name"].str.endswith("monthly") == True])
    solar_filter4 = pd.DataFrame(solar_filter3[solar_filter3["name"].str.contains("all primemovers") == True])    
    # Append each plant's list of series to the master all series dataframe
    temp_series_df = solar_df.append(solar_filter4, ignore_index = True)
    solar_df = temp_series_df
    
solar_df

,name,series_id
0,"Net generation : River Bend Solar, LLC (60058) : solar : all primemovers : monthly",ELEC.PLANT.GEN.60058-SUN-ALL.M
1,Net generation : Ocotillo (116) : solar : all primemovers : monthly,ELEC.PLANT.GEN.116-SUN-ALL.M
2,Net generation : Saguaro (118) : solar : all primemovers : monthly,ELEC.PLANT.GEN.118-SUN-ALL.M
3,Net generation : Agua Fria (141) : solar : all primemovers : monthly,ELEC.PLANT.GEN.141-SUN-ALL.M
4,Net generation : Springerville (8223) : solar : all primemovers : monthly,ELEC.PLANT.GEN.8223-SUN-ALL.M
5,Net generation : Prescott Airport (56228) : solar : all primemovers : monthly,ELEC.PLANT.GEN.56228-SUN-ALL.M
6,Net generation : Maricopa Solar (57140) : solar : all primemovers : monthly,ELEC.PLANT.GEN.57140-SUN-ALL.M
7,Net generation : Agua Caliente Solar Project (57373) : solar : all primemovers : monthly,ELEC.PLANT.GEN.57373-SUN-ALL.M
8,Net generation : UASTP I (57392) : solar : all primemovers : monthly,ELEC.PLANT.GEN.57392-SUN-ALL.M
9,Net generation : Roger Road WWTP (57524) : solar : all primemovers : monthly,ELEC.PLANT.GEN.57524-SUN-ALL.M


In [57]:
# Fix plant name
solar_name_temp1 = solar_df["name"].str.split(':').str.get(1)
solar_name_temp2 = pd.DataFrame(solar_name_temp1.str.split('(').str.get(0), columns = ["name"])
solar_final_df = pd.concat([solar_df["series_id"], solar_name_temp2], axis=1)
solar_final_df


,series_id,name
0,ELEC.PLANT.GEN.60058-SUN-ALL.M,"River Bend Solar, LLC"
1,ELEC.PLANT.GEN.116-SUN-ALL.M,Ocotillo
2,ELEC.PLANT.GEN.118-SUN-ALL.M,Saguaro
3,ELEC.PLANT.GEN.141-SUN-ALL.M,Agua Fria
4,ELEC.PLANT.GEN.8223-SUN-ALL.M,Springerville
5,ELEC.PLANT.GEN.56228-SUN-ALL.M,Prescott Airport
6,ELEC.PLANT.GEN.57140-SUN-ALL.M,Maricopa Solar
7,ELEC.PLANT.GEN.57373-SUN-ALL.M,Agua Caliente Solar Project
8,ELEC.PLANT.GEN.57392-SUN-ALL.M,UASTP I
9,ELEC.PLANT.GEN.57524-SUN-ALL.M,Roger Road WWTP


### ER Diagram: Generation and lat/lon
This dataframe can populate the Generation table (though that table still needs an index) and can also populate the latitude and longitude of the Solar Location table

Finally, for each series in the solar_loc_df dataframe, collect the remaining data needed and put it into another dataframe:
Series ID
lat
lon
month
year
MWH
AND: take out the MWH that are zero and make those NA, because zero production should not be factored into any training data.

In [111]:
# Create empty dataframe to hold electricity generation data
columns = ["series_id", "lat", "lon", "latlon", "year", "month", "mwh"]
gen_final_df = pd.DataFrame(columns = columns)

In [119]:
# Iterate over each series in the solar power dataframe
for series in solar_final_df.itertuples():
    series_id = str(series[1])
    # Download the plant data
    gen_url = 'http://api.eia.gov/series/?api_key=' + EIA_API_KEY + '&series_id=' + series_id
    gen = requests.get(gen_url)
    # Put downloaded data into a dataframe
    gen_dict = gen.json()
    gen_json = pd.io.json.json_normalize(gen_dict['series'])
    gen_data_df = pd.DataFrame(gen_json['data'][0], columns = ["date", 'mwh'])
    # Organize data into the right columns in the dataframe
    gen_data_df['year'] = gen_data_df['date'][0][:4]
    gen_data_df['month'] = gen_data_df['date'][0][4:]
    gen_data_df['series_id'] = np.where(gen_data_df["mwh"] == 0, '', gen_json['series_id'])
    gen_data_df['lat'] = np.where(gen_data_df["mwh"] == 0, '', gen_json['lat'])
    gen_data_df['lon'] = np.where(gen_data_df["mwh"] == 0, '', gen_json['lon'])
    gen_data_df['latlon'] = np.where(gen_data_df["mwh"] == 0, '', gen_json['latlon'])
    # Remove all rows with zero electricty generation
    temp1_gen_df = gen_data_df[gen_data_df["mwh"] != 0]
    # Append each series's monthly electricity generation to the generation dataframe
    temp2_gen_df = gen_final_df.append(temp1_gen_df, ignore_index = True)
    gen_final_df = temp2_gen_df

gen_final_df


,date,lat,latlon,lon,month,mwh,series_id,year
0,201609,34.831511,"34.831511,-87.842204",-87.842204,09,467.000,ELEC.PLANT.GEN.60058-SUN-ALL.M,2016
1,201609,33.4225,"33.4225,-111.9122",-111.9122,09,56.000,ELEC.PLANT.GEN.116-SUN-ALL.M,2016
2,201608,33.4225,"33.4225,-111.9122",-111.9122,09,64.000,ELEC.PLANT.GEN.116-SUN-ALL.M,2016
3,201607,33.4225,"33.4225,-111.9122",-111.9122,09,70.000,ELEC.PLANT.GEN.116-SUN-ALL.M,2016
4,201606,33.4225,"33.4225,-111.9122",-111.9122,09,70.000,ELEC.PLANT.GEN.116-SUN-ALL.M,2016
5,201605,33.4225,"33.4225,-111.9122",-111.9122,09,82.000,ELEC.PLANT.GEN.116-SUN-ALL.M,2016
6,201604,33.4225,"33.4225,-111.9122",-111.9122,09,64.000,ELEC.PLANT.GEN.116-SUN-ALL.M,2016
7,201603,33.4225,"33.4225,-111.9122",-111.9122,09,57.000,ELEC.PLANT.GEN.116-SUN-ALL.M,2016
8,201602,33.4225,"33.4225,-111.9122",-111.9122,09,46.000,ELEC.PLANT.GEN.116-SUN-ALL.M,2016
9,201601,33.4225,"33.4225,-111.9122",-111.9122,09,33.000,ELEC.PLANT.GEN.116-SUN-ALL.M,2016


## Load data into Postgres

I created a Jupyter notebook on an AMI for writing code to load Postgres tables